# Scraping and storing all pdfs related to "Eidgenössische Steuerverwaltung"

In [6]:
import urllib
import requests
import time
from bs4 import BeautifulSoup
from re import compile, sub, match, findall

In [7]:
url = "https://www.admin.ch/opc/de/federal-gazette/2015/index_32.html"

On this website you can find pdfs of the The Federal Gazette, which is a publication of the Swiss government. The pdfs related to "Eidgenössische Steuerverwaltung" and "Amtshilfe" contain names of foreign people, which are asked to get in touch immediately with the Swiss government. It is probably a matter of tax fraud.


In [9]:
pdflinks = []

url = "https://www.admin.ch/opc/de/federal-gazette/2015/index_32"
page = BeautifulSoup(requests.get(url,headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:5.0)","Referer":"http://google.com"}).text )

for year in range(2011,2016):
    for week in range(1,53):
        url = "https://www.admin.ch/opc/de/federal-gazette/"+str(year)+"/index_"+str(week)
        page = BeautifulSoup(requests.get(url,headers={}).text )

        table = page.find("table","table table-striped")
        if table:
            trs = table.find_all("tr")
            for row in trs:
                doclink = row.find("td", attrs = {"class":"nowrap"})
                if doclink:
                    title = doclink.find_next_sibling("td")
                    if title:
                        if "estv" in title.getText().lower() and "amtshilfe" in title.getText().lower():
                            link = doclink.find("a")
                            filelink = "http://www.admin.ch"+urllib.unquote(link.get("href")).replace("&amp;","&")
                            filename = link.text
                            print filelink

                            #urllib.urlretrieve(filelink,"steuer/"+filename+".pdf") BLOCKED by the Swiss gov

                            pdflinks.append(filelink)
                            
                            #time.sleep(15)

http://www.admin.ch/opc/de/federal-gazette/2011/8765.pdf
http://www.admin.ch/opc/de/federal-gazette/2012/8005.pdf
http://www.admin.ch/opc/de/federal-gazette/2012/8032.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/4850.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/4857.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/8399.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/1661.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/1792.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/2022.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/2363.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/3307.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/3748.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/3750.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/3751.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/4132.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/4133.pdf
http://www.admin.ch/opc/de/federal-gazette/2014/5234.pdf
http://www.admin.ch/opc/de/fede

# New try with WGET

The Swiss gov blocked our scraper, ``urllib.urlretrieve`` didn't store real pdfs. Instead we got a message from the Swiss Gov: 

*"Dear Visitor
The federal authorities of Switzerland make use of a system scanning website requests for security violations. Your request has been flagged as such a violation and has therefore been blocked. It is possible that this constitutes a false positive. If you have received this message in error, please contact the service desk of the Federal Office for IT and Telecommunications, stating support-id D-3873271559171869965 as well as any additional information which may be relevant to the case. We will evaluate your case and report back to you within a week.
Federal Office of IT and Telecommunications"*

So, let's try WGET.

In [10]:
# ! brew install wget

In [ ]:
for pdflink in pdflinks:
    !wget -nc --random-wait -P steuer {pdflink}

--2015-08-24 16:51:40--  http://www.admin.ch/opc/de/federal-gazette/2011/8765.pdf
Resolving www.admin.ch... 162.23.128.199
Connecting to www.admin.ch|162.23.128.199|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.admin.ch/opc/de/federal-gazette/2011/8765.pdf [following]
--2015-08-24 16:51:40--  https://www.admin.ch/opc/de/federal-gazette/2011/8765.pdf
Connecting to www.admin.ch|162.23.128.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135530 (132K) [application/pdf]
Saving to: 'steuer/8765.pdf'

8765.pdf            100%[=====================>] 132.35K   428KB/s   in 0.3s   

2015-08-24 16:51:41 (428 KB/s) - 'steuer/8765.pdf' saved [135530/135530]

--2015-08-24 16:51:41--  http://www.admin.ch/opc/de/federal-gazette/2012/8005.pdf
Resolving www.admin.ch... 162.23.128.199
Connecting to www.admin.ch|162.23.128.199|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.admin.ch/opc/de/

Hooray! WGET stores all the pdfs we want in our local folder.

In [4]:
# Next step: parsing > pdf to text